In [25]:
#r "nuget:XPlot.Plotly"
#r "nuget:Microsoft.Data.Analysis"
#r "nuget:XPlot.Plotly.Interactive"

Installed Packages Microsoft.Data.Analysis, 0.20.1 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

In [26]:
using XPlot.Plotly;
using Microsoft.Data.Analysis;
using System.Linq;
using System.Threading;
using XPlot.Plotly.Interactive;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;

In [27]:
var rnd = new Random();

In [28]:
public enum CollapseEvent {
  Scattering = 0,
  Absorption = 1,
  Fission = 2,
  OutOfRange = 3
}

In [29]:
public class Atom 
{
  public int X { get; set; }
  public int Y { get; set; }
  public int U { get; set; }
  public int V { get; set; }
  public int L { get; set; }
  public int CurrentL { get; set; }
  public double Lambda { get; set;}
  public int Border { get; set; }

  public Atom(int x, int y, double lambda, Random rnd, int border) {
    CurrentL = 0;
    L = Convert.ToInt32(- Math.Log(rnd.NextDouble()) / lambda);
    X = x;
    Y = y;
    U = rnd.Next(2) == 0 ? -1 : 1;
    V = rnd.Next(2) == 0 ? -1 : 1;
    Lambda = lambda;
    Border = border;
  }

  public (IEnumerable<Atom> Atoms, CollapseEvent? Event) Collapse(Random rnd) {
    var collapseEvent = (CollapseEvent)rnd.Next(0, 3);

    switch (collapseEvent) {
      case CollapseEvent.Scattering:
        return (new List<Atom>() { new Atom(X, Y, Lambda, rnd, Border) }, collapseEvent);
      case CollapseEvent.Absorption:
        return (new List<Atom>(), collapseEvent);
      case CollapseEvent.Fission:
        return (new List<Atom>() { new Atom(X, Y, Lambda, rnd, Border), new Atom(X, Y, Lambda, rnd, Border) }, collapseEvent);
    }

    return (new List<Atom>() { this }, null);
  }

  public (IEnumerable<Atom> Atoms, CollapseEvent? Event) Step(Random rnd) {
    CurrentL++;
    X += U;
    Y += V;

    if (X < 0 || Y < 0 || X > Border || Y > Border) {
      return (new List<Atom>(), CollapseEvent.OutOfRange);
    }

    if (L == CurrentL) {
      return Collapse(rnd);
    }
    
    return (new List<Atom>() { this }, null);
  }
}

In [30]:
var lmd = 0.1;
var borderLen = 50;
var atoms = new List<Atom>() { new Atom(10, 10, lmd, rnd, borderLen), new Atom(25, 25, lmd, rnd, borderLen),new Atom(20, 20, lmd, rnd, borderLen), new Atom(30, 30, lmd, rnd, borderLen), new Atom(10, 40, lmd, rnd, borderLen) };

In [31]:
var border1 = new Scatter
{   
    x = new List<int>() { 0, borderLen, borderLen, 0, 0 },
    y = new List<int>() { 0, 0, borderLen, borderLen, 0 },
    mode = "line",
    marker = new Marker 
    { 
        color = "red",
        size = 6,
        line = new Line {
            color = "white",
            width = 1
        }
    }
};

var layout = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = "x"
    },
    yaxis = new Yaxis
    {
        title = "y"
    },
    title = $"Generation #0",    
};

var plot = Chart.Plot(border1, layout);
plot.Height = 700;
plot.Width = 700;

var animationOutput = display(plot);

var time = new List<int>();
var atomsCounts = new List<int>();
var scatterings = new List<int>();
var absorptions = new List<int>();
var fissions = new List<int>();
var outOfRanges = new List<int>();

for (int i = 0; i < 100; i++){
    if(atoms.Count() == 0){
        break;
    }
    var copyAtoms = atoms.ToList();
    atoms = new List<Atom>();
    var scattering = 0;
    var absorption = 0;
    var fission = 0; 
    var outOfRange = 0;
    foreach (var atom in copyAtoms) {
        var newAtoms = atom.Step(rnd);
        atoms.AddRange(newAtoms.Atoms);
        switch (newAtoms.Event) {
            case CollapseEvent.Scattering:
                scattering++;
                break;
            case CollapseEvent.Absorption:
                absorption++;
                break;
            case CollapseEvent.Fission:
                fission++;
                break;
            case CollapseEvent.OutOfRange:
                outOfRange++;
                break;
        }
    } 
    time.Add(i);
    atomsCounts.Add(atoms.Count());   
    scatterings.Add(scattering);
    absorptions.Add(absorption);
    fissions.Add(fission);
    outOfRanges.Add(outOfRange);

    var scatter = new Scatter
    {   
        x = atoms.Select(a => a.X),
        y = atoms.Select(a => a.Y),
        mode = "markers",
        marker = new Marker 
        { 
            color = "blue",
            size = 15,
            line = new Line {
                color = "white",
                width = 1
            }
        }
    };

    layout.title = $"Generation #{i} (atoms: {atoms.Count()})";

    var plot = Chart.Plot(new List<Trace>() { scatter, border1 }, layout);
    plot.Height = 800;
    plot.Width = 800;

    animationOutput.Update(plot);
    Thread.Sleep(1000);
}


<!DOCTYPE html>

In [32]:
var atomsCountScatter = new Scatter
{   
    x = time,
    y = atomsCounts,
    mode = "lines",
    marker = new Marker 
    { 
        color = "black",
        size = 1
    },
    name = "Число атомов"
};

var scatteringsCountScatter = new Scatter
{   
    x = time,
    y = scatterings,
    mode = "lines",
    marker = new Marker 
    { 
        color = "red",
        size = 1
    },
    name = "Рассеились"
};

var absorptionsCountScatter = new Scatter
{   
    x = time,
    y = absorptions,
    mode = "lines",
    marker = new Marker 
    { 
        color = "blue",
        size = 1
    },
    name = "Поглотились"
};

var fissionsCountScatter = new Scatter
{   
    x = time,
    y = fissions,
    mode = "lines",
    marker = new Marker 
    { 
        color = "green",
        size = 1
    },
    name = "Расщепились"
};

var outOfRangesCountScatter = new Scatter
{   
    x = time,
    y = fissions,
    mode = "lines",
    marker = new Marker 
    { 
        color = "magenta",
        size = 1
    },
    name = "Вышло за пределы"
};

var atomsCountLayout = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = "time"
    },
    yaxis = new Yaxis
    {
        title = "count"
    },
    title = $"Count of atoms",    
};

var plot = Chart.Plot(new List<Trace>() { atomsCountScatter, scatteringsCountScatter, absorptionsCountScatter, fissionsCountScatter, outOfRangesCountScatter }, atomsCountLayout);
plot.Height = 1000;
plot.Width = 1000;

plot

<!DOCTYPE html>